<a href="https://colab.research.google.com/github/Ihoold/pixelgen/blob/master/funit4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from unittest import mock
import os
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from tensorflow.python.estimator import estimator
from google.colab import drive, auth

In [0]:
R, C = 4, 3
EXAMPLES = R * C
    
CHANNELS = 4
LATENT_DIM = 128
ADD_NOISE_TO_EXAMPLE = False

DROPOUT_PROB = 0.5
ALPHA = 0.2
BATCH_SIZE = 1024
EPOCHS = 100000
EVAL_EPOCHS = 1000
G_LR = 0.0002
D_LR = 0.0001
KERNEL_SIZE = 4
NUM_CLASSES = 21

MODEL_NAME = 'FUNIT'
RUN_NAME = 'FUNIT_only_gen_9'

data_train_file = 'gs://tputestingmnist/datasets/dataset_train_funit.tfrecords'
data_test_file = 'gs://tputestingmnist/datasets/dataset_test_funit.tfrecords'
MODEL_DIR = 'gs://tputestingmnist/{}/{}/'.format(MODEL_NAME, RUN_NAME)
# MODEL_DIR = 'gs://tputestingmnist/{}/'.format(RUN_NAME)
GOOGLE_DRIVE_DIR = '/content/gdrive/My Drive/Programowanie/PixelGen/{}'.format(RUN_NAME)
TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

In [0]:
#################################### SETUP #####################################

def setup():
    drive.mount('/content/gdrive')
    auth.authenticate_user()


def upload_credentials():
    # Upload credentials to TPU.
    with tf.Session(TF_MASTER) as sess:    
        with open('/content/adc.json', 'r') as f:
            auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)

In [0]:
################################# DATA INPUT ###################################

def parser(serialized_example):
        """Parses a single example into image and label tensors."""
        features = tf.parse_single_example(
            serialized_example,
            features={
                'image_content': tf.FixedLenFeature([], tf.string),
                'image_style': tf.FixedLenFeature([], tf.string),
                'label': tf.FixedLenFeature([], tf.int64) 
            })

        content_image = tf.decode_raw(features['image_content'], tf.uint8)
        content_image.set_shape([48 * 48 * 4])
        content_image = tf.reshape(content_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        content_image = tf.cast(content_image, dtype=tf.float32) / 127.5 - 1

        style_image = tf.decode_raw(features['image_style'], tf.uint8)
        style_image.set_shape([48 * 48 * 4])
        style_image = tf.reshape(style_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        style_image = tf.cast(style_image, dtype=tf.float32) / 127.5 - 1
        
        label = features['label']

        return {'style_images': style_image,
                'content_images': content_image}, label


def make_input_fn(is_training=True, one_batch=False):
    def input_fn(params):
        batch_size = params['batch_size']
        dataset = tf.data.TFRecordDataset(data_train_file, buffer_size=8*1024*1024)
        dataset = dataset.map(parser).cache().shuffle(batch_size)
        if is_training:
            dataset = dataset.repeat()
        if one_batch:
            dataset = dataset.take(params['batch_size'])
            
        dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(8)

        return dataset
    return input_fn


def predict_input_fn(params):
    batch_size = params['batch_size']
    dataset = tf.data.TFRecordDataset(data_test_file).map(parser).cache().shuffle(batch_size)
    style_images, content_images, _ = dataset.prefetch(batch_size).batch(batch_size, drop_remainder=True).make_one_shot_iterator().get_next()
    
    features = {
        'style_images': style_images,
        'content_images': content_images,
    }
    return features, None


In [0]:
############################### DATA SAVEING ###################################
 
def images_to_zero_one(images):
        return np.clip(np.array(images) * 0.5 + 0.5, 0., 1.)


def save_imgs(epoch, images):
    if not os.path.exists(GOOGLE_DRIVE_DIR):
        os.mkdir(GOOGLE_DRIVE_DIR)

    # Rescale images to 0 - 1
    images = images_to_zero_one(images)
    fig, axs = plt.subplots(R, C, figsize=(20,20))

    for i in range(R):
        for j in range(C):
            axs[i,j].imshow(images[C*i + j])
            axs[i,j].axis('off')

    fig.savefig(os.path.join(GOOGLE_DRIVE_DIR, '{}.png'.format(epoch)))
    plt.close()

In [0]:
################################## MODEL #######################################

def _relu(x):
    return tf.nn.relu(x)


def _leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=ALPHA)


def _batch_norm(x, is_training, name):
    return tf.layers.batch_normalization(x, momentum=0.8, epsilon=1e-5, 
                                         training=is_training, name=name)

def _make_batch_norm(is_training, name):
    def bn(x):
        return _batch_norm(x, is_training=is_training, name=name)
    return bn

def _instance_norm(x):
    return tf.contrib.layers.instance_norm(x)


def _make_adain_norm(style_mean, style_var):
    def _adain_norm(x):
        mean, var = tf.nn.moments(x, axes=[0])
        return tf.nn.batch_normalization(x, mean, var, style_mean, style_var)
    return _adain_norm


def _dense(x, neurons, name, activation=None):
    return tf.layers.dense(x, neurons, name=name, activation=activation,
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def _conv2d(x, filters, kernel_size, stride, name, activation=None):
    return tf.layers.conv2d(x, filters, [kernel_size, kernel_size], 
                            strides=[stride, stride], activation=activation,
                            padding='same', name=name,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

def _pooling(x, kernel, stride, index):
    return tf.layers.average_pooling2d(x, pool_size=kernel, strides=stride, name='pool_{}'.format(index))

def _deconv2d(x, filters, stride, name, activation=None):
    return tf.layers.conv2d_transpose(x, filters, [KERNEL_SIZE, KERNEL_SIZE],
                                      strides=[stride, stride], activation=activation,
                                      padding='same', name=name,
                                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def convolution_block(x, filters, kernel_size, resize_factor, index, activation=_relu, activation_first=False, normalization=None):
    if activation and activation_first:
        x = activation(x)
    x = _conv2d(x, kernel_size=kernel_size, filters=filters, stride=resize_factor, activation=None, name='conv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation and not activation_first:
        x = activation(x)
    return x


def deconvolution_block(x, filters, resize_factor, index, activation=_relu, normalization=None):
    x = _deconv2d(x, filters=filters, stride=resize_factor, activation=None, name='deconv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def fully_connected_block(x, neurons, index, activation=_relu, normalization=None):
    x = _dense(x, neurons=neurons, activation=None, name='fc_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def residual_block(x, filters_in, filters_out, index, kernel_size=3, activation=_relu, activation_first=False, normalization=None):
    forwarded_x = convolution_block(x, filters_in, kernel_size, resize_factor=1, 
                                    index='res_con1_{}'.format(index), 
                                    activation_first=activation_first, activation=activation)
    
    forwarded_x = convolution_block(forwarded_x, filters_out, kernel_size, resize_factor=1, 
                                    index='res_con2_{}'.format(index), 
                                    activation_first=activation_first, activation=None)
    
    if filters_in != filters_out:
        x = convolution_block(x, filters_out, kernel_size=1, resize_factor=1, index='res_con3_{}'.format(index), activation=None)
    x = tf.add(forwarded_x, x)
    return x


def adain_residual_block(x, mean, var, filters_in, filters_out, index, activation=_relu, activation_first=False):
    with tf.variable_scope('adain_{}'.format(index)):
        normalization = _make_adain_norm(mean, var)
        return residual_block(x, filters_in, filters_out, index, 3, activation, activation_first, normalization)


def content_encoder(x):
    x = convolution_block(x, 64, kernel_size=7, resize_factor=1, normalization=_instance_norm, index='con1')
    x = convolution_block(x, 128, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con2')
    x = convolution_block(x, 256, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con3')
    x = convolution_block(x, 512, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con4')
    
    x = residual_block(x, 512, 512, index='res1', normalization=_instance_norm)
    x = residual_block(x, 512, 512, index='res2', normalization=_instance_norm)
    return x


def class_encoder(x):
    x = convolution_block(x, 64, kernel_size=7, resize_factor=1, index='cls1')
    x = convolution_block(x, 128, kernel_size=4, resize_factor=2, index='cls2')
    x = convolution_block(x, 256, kernel_size=4, resize_factor=2, index='cls3')
    x = convolution_block(x, 512, kernel_size=4, resize_factor=2, index='cls4')
    x = convolution_block(x, 1024, kernel_size=4, resize_factor=2, index='cls5')
    x = _pooling(x, 3, 3, index='cls6')
    return x


def decoder(content_latent):    
    class_latent = fully_connected_block(class_latent, 256, 'dec11')
    class_latent = fully_connected_block(class_latent, 256, 'dec12')
    class_latent = fully_connected_block(class_latent, 256, 'dec13')
    mean, var = tf.nn.moments(class_latent, axes=[0])
    
    x = adain_residual_block(content_latent, mean, var, 512, 512, index='res3', activation=None)
    x = adain_residual_block(x, mean, var, 512, 512, index='res4', activation=None)

    x = deconvolution_block(content_latent, 512, resize_factor=1, normalization=_instance_norm, index='dec0')
    x = deconvolution_block(x, 256, resize_factor=2, normalization=_instance_norm, index='dec1')
    x = deconvolution_block(x, 128, resize_factor=2, normalization=_instance_norm, index='dec2')
    x = deconvolution_block(x, 64, resize_factor=2, normalization=_instance_norm, index='dec3')
    x = deconvolution_block(x, CHANNELS, resize_factor=1, normalization=_instance_norm, index='dec4', activation=tf.tanh)
    
    return x


class Funit:

    @staticmethod
    def discriminator(x, label, scope='Discriminator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            x = convolution_block(x, 64, kernel_size=7, resize_factor=1, index='disc_11')
            
            x = residual_block(x, 64, 64, 'disc_12', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 64, 128, 'disc_13', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_14')
            
            x = residual_block(x, 128, 128, 'disc_15', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 128, 256, 'disc_16', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_17')
            
            x = residual_block(x, 256, 256, 'disc_18', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 256, 512, 'disc_19', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_20')
            
            x = residual_block(x, 512, 512, 'disc_21', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 512, 1024, 'disc_22', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_23')
            
            x = residual_block(x, 1024, 1024, 'disc_24', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 1024, 1024, 'disc_25', kernel_size=3, activation=_leaky_relu, activation_first=True)
            
            x = _leaky_relu(x)
            x = tf.layers.flatten(x)
            x = _dense(x, NUM_CLASSES, 'disc_out')
            label = tf.expand_dims(label, axis=1)
            x = tf.gather_nd(x, label, batch_dims=1)
            
            return x
          
    @staticmethod
    def generator(image_content, image_style, is_training, scope='Generator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
#             class_latent = class_encoder(image_style)
            
#             x = decoder(content_latent)

            x = content_encoder(image_style)
        
            x = deconvolution_block(x, 512, resize_factor=2, index='deconv21', normalization=_make_batch_norm(is_training, 'bn5'))
            x = deconvolution_block(x, 256, resize_factor=2, index='deconv22', normalization=_make_batch_norm(is_training, 'bn6'))
            x = deconvolution_block(x, 128, resize_factor=2, index='deconv23', normalization=_make_batch_norm(is_training, 'bn7'))
            x = deconvolution_block(x, CHANNELS, resize_factor=1, index='deconv24', activation=tf.tanh, normalization=None)
            return x

In [0]:
################################ MODEL FUN #####################################
def make_model_fn(model):

    def model_fn(features, labels, mode, params):
        with tf.variable_scope('inputs'):
            content_images = features['content_images']
            style_images = features['style_images']
        generated_images = model.generator(content_images, style_images, (mode == tf.estimator.ModeKeys.TRAIN))
        
        # PREDICT #
        if mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'generated_images': generated_images,
                'content_images': content_images,
                'style_images': style_images
            }
            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions=predictions)

        # Discriminator loss
        d_on_data_logits = tf.squeeze(model.discriminator(style_images, labels))
        d_on_g_logits = tf.squeeze(model.discriminator(generated_images, labels))

        
        with tf.variable_scope('losses'):
            d_loss = tf.contrib.gan.losses.wargs.modified_discriminator_loss(
                discriminator_real_outputs=d_on_data_logits,
                discriminator_gen_outputs=d_on_g_logits,
                reduction=tf.losses.Reduction.NONE,
                label_smoothing=0.2
            )
            d_loss_reduced = tf.reduce_mean(d_loss)

            # Generator loss
            g_loss = tf.contrib.gan.losses.wargs.modified_generator_loss(
                discriminator_gen_outputs=d_on_g_logits,
                reduction=tf.losses.Reduction.NONE
            )
            g_loss_reduced = tf.reduce_mean(g_loss)
            
        # TRAIN #
        if mode == tf.estimator.ModeKeys.TRAIN:
            
            with tf.variable_scope('optimizer'):
                d_optimizer = tf.train.AdamOptimizer(learning_rate=D_LR, beta1=0.5)
                d_optimizer = tf.contrib.tpu.CrossShardOptimizer(d_optimizer)
            
                g_optimizer = tf.train.AdamOptimizer(learning_rate=G_LR, beta1=0.5)
                g_optimizer = tf.contrib.tpu.CrossShardOptimizer(g_optimizer)
         
                with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                    d_step = d_optimizer.minimize(d_loss_reduced, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                     scope='Discriminator'))
                    g_step = g_optimizer.minimize(g_loss_reduced, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                     scope='Generator'))
                    increment_step = tf.assign_add(tf.train.get_or_create_global_step(), 1)
                    joint_op = tf.group([d_step, g_step, increment_step])

                    a = tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=d_loss_reduced+g_loss_reduced, train_op=joint_op)
                    return a

        # EVAL #
        elif mode == tf.estimator.ModeKeys.EVAL:
            def _eval_metric_fn(d_loss, g_loss, d_real_labels, d_gen_lanels, d_real_logits, d_gen_logits):
                return {
                    'discriminator_loss': tf.metrics.mean(d_loss),
                    'generator_loss': tf.metrics.mean(g_loss),
                    'discriminator_real_accuracy': tf.metrics.accuracy(labels=d_real_labels, predictions=tf.math.round(tf.sigmoid(d_real_logits))),
                    'discriminator_gen_accuracy': tf.metrics.accuracy(labels=d_gen_lanels, predictions=tf.math.round(tf.sigmoid(d_gen_logits))),
                }

            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=d_loss_reduced + g_loss_reduced,
                                                   eval_metrics=(_eval_metric_fn, [d_loss, g_loss, 
                                                                                   tf.ones_like(labels), tf.zeros_like(labels),
                                                                                   d_on_data_logits, d_on_g_logits]))
    return model_fn

In [0]:
################################ ESTIMATORS ####################################

def make_estimators(model, only_cpu=False):
    model_fn = make_model_fn(model)
    
    config = tf.contrib.tpu.RunConfig(
        master=TF_MASTER,
        save_checkpoints_steps=EVAL_EPOCHS,
        save_checkpoints_secs=None,
        save_summary_steps=EVAL_EPOCHS,
        model_dir=MODEL_DIR,
        keep_checkpoint_max=3,
        tpu_config=tf.contrib.tpu.TPUConfig(iterations_per_loop=EVAL_EPOCHS))

    if not only_cpu:
        # TPU-based estimator used for TRAIN and EVAL
        est = tf.contrib.tpu.TPUEstimator(
            model_fn=model_fn,
            use_tpu=True,
            config=config,
            train_batch_size=BATCH_SIZE,
            eval_batch_size=BATCH_SIZE)
    else:
        est = None

    # CPU-based estimator used for PREDICT (generating images)
    cpu_est = tf.contrib.tpu.TPUEstimator(
        model_fn=model_fn,
        use_tpu=False,
        config=config,
        predict_batch_size=EXAMPLES)
    
    return est, cpu_est

In [0]:
################################# TRAINING #####################################

def train(est, cpu_est):
    current_step = estimator._load_global_step_from_checkpoint_dir(MODEL_DIR)
    print('Starting training')

    while current_step < EPOCHS:
        next_checkpoint = int(min(current_step + EVAL_EPOCHS, EPOCHS))
        est.train(input_fn=make_input_fn(), max_steps=next_checkpoint)
        current_step = next_checkpoint
        print('Finished training step %d' % current_step)

        # Evaluation
        metrics = est.evaluate(input_fn=make_input_fn(False), steps=1)
        print('Finished evaluating')
        print(metrics)

        # Render some generated images
        generated_iter = cpu_est.predict(input_fn=make_input_fn(False, one_batch=True))
        images = [np.concatenate([p['content_images'], p['style_images'], p['generated_images']], axis=1) for p in generated_iter]
        save_imgs(str(current_step), images)
        print('Finished generating images')

In [0]:
def do_experiment():
    setup()
    upload_credentials()
    model = Funit()
    est, cpu_est = make_estimators(model)
    train(est, cpu_est)

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

try:
    do_experiment()
except Exception as e:
    print (e)
    pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


W0818 09:56:37.367154 140270304130944 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0818 09:56:39.572362 140270304130944 estimator.py:1984] Estimator's model_fn (<function make_model_fn.<locals>.model_fn at 0x7f92f203ac80>) includes params argument, but params are not passed to Estimator.
I0818 09:56:39.575609 140270304130944 estimator.py:209] Using config: {'_model_dir': 'gs://tputestingmnist/FUNIT/FUNIT_only_gen_9/', '_tf_random_seed': None, '_save_summary_steps': 1000, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  

Starting training


I0818 09:56:41.035561 140270304130944 estimator.py:1145] Calling model_fn.
W0818 09:56:41.179435 140270304130944 deprecation.py:323] From <ipython-input-6-7644f9e48ff3>:39: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0818 09:56:41.947423 140270304130944 deprecation.py:323] From <ipython-input-6-7644f9e48ff3>:48: conv2d_transpose (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
W0818 09:56:42.657723 140270304130944 deprecation.py:323] From <ipython-input-6-7644f9e48ff3>:12: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should

Finished training step 1000


...

I0818 10:50:27.324369 140270304130944 estimator.py:1147] Done calling model_fn.
I0818 10:50:27.358446 140270304130944 evaluation.py:255] Starting evaluation at 2019-08-18T10:50:27Z
I0818 10:50:27.359721 140270304130944 tpu_estimator.py:499] TPU job name tpu_worker
I0818 10:50:27.668324 140270304130944 monitored_session.py:240] Graph was finalized.
I0818 10:50:27.727010 140270304130944 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_gen_9/model.ckpt-8000
I0818 10:50:31.348528 140270304130944 session_manager.py:500] Running local_init_op.
I0818 10:50:31.461600 140270304130944 session_manager.py:502] Done running local_init_op.
I0818 10:50:31.684621 140270304130944 tpu_estimator.py:557] Init TPU system
I0818 10:50:39.694814 140270304130944 tpu_estimator.py:566] Initialized TPU in 8 seconds
I0818 10:50:39.696904 140268921198336 tpu_estimator.py:514] Starting infeed thread controller.
I0818 10:50:39.697604 140268912805632 tpu_estimator.py:533] Starting outfeed

Finished evaluating
{'discriminator_gen_accuracy': 0.953125, 'discriminator_loss': 98.16623, 'discriminator_real_accuracy': 0.75878906, 'generator_loss': 50.729694, 'loss': 148.4605, 'global_step': 8000}


I0818 10:50:50.884531 140270304130944 estimator.py:1147] Done calling model_fn.
I0818 10:50:51.421042 140270304130944 monitored_session.py:240] Graph was finalized.
I0818 10:50:51.483553 140270304130944 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_gen_9/model.ckpt-8000
I0818 10:50:51.776803 140270304130944 session_manager.py:500] Running local_init_op.
I0818 10:50:51.793874 140270304130944 session_manager.py:502] Done running local_init_op.
I0818 10:50:52.206763 140270304130944 error_handling.py:96] prediction_loop marked as finished
I0818 10:50:52.208063 140270304130944 error_handling.py:96] prediction_loop marked as finished


Finished generating images


I0818 10:50:53.265094 140270304130944 estimator.py:1145] Calling model_fn.
I0818 10:51:00.857346 140270304130944 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0818 10:51:01.037755 140270304130944 estimator.py:1147] Done calling model_fn.
I0818 10:51:01.039587 140270304130944 tpu_estimator.py:499] TPU job name tpu_worker
I0818 10:51:01.758948 140270304130944 monitored_session.py:240] Graph was finalized.
I0818 10:51:01.985038 140270304130944 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_gen_9/model.ckpt-8000
I0818 10:51:06.659628 140270304130944 session_manager.py:500] Running local_init_op.
I0818 10:51:07.026427 140270304130944 session_manager.py:502] Done running local_init_op.
I0818 10:51:12.159782 140270304130944 basic_session_run_hooks.py:606] Saving checkpoints for 8000 into gs://tputestingmnist/FUNIT/FUNIT_only_gen_9/model.ckpt.
I0818 10:51:26.005724 140270304130944 util.py:98] Initialized dataset iterators in 0 seconds
I0818 10:51: